# NLP Classifier
By : Lana Fuadi
<br>LinkedIn : https://www.linkedin.com/in/lanafuadi/
<br>GitHub : https://github.com/lanafuadi

Ini adalah project percobaan untuk membuat sentiment analysis dengan mengimplementasikan Natural Language Processing (NLP) dan mengklasifikasikan sebuah komentartermasuk komentar positif atau negatif.

In [11]:
import nltk
import pickle
import re
from collections import defaultdict
from math import log

In [17]:
# Membuat fungsi training dengan classifier NaiveBayes
def nb_train(positive_data, negative_data):
    positive_data        = [text.lower() for text in positive_data] # untuk mengubah huruf besar menjadi huruf kecil
    negative_data        = [text.lower() for text in negative_data] # untuk mengubah huruf besar menjadi huruf kecil
    total_positive_docs  = len(positive_data) # menentukan jumlah teks dari kalimat positif
    total_negative_docs  = len(negative_data) # menentukan jumlah teks dari kalimat negative
    total_docs           = total_positive_docs + total_negative_docs # total teks postive ditambah negative
    prior_positive       = total_positive_docs / total_docs # menghitung probabilitas munculnya kata positif
    prior_negative       = total_negative_docs / total_docs # menghitung probabilitas munculnya kata positif
    word_freq_positive   = defaultdict(int) # menambah frequency ke dalam dictionary default
    word_freq_negative   = defaultdict(int) # menambah frequency ke dalam dictionary default
    total_words_positive = 0 # jumlah awal kata positif
    total_words_negative = 0 # jumlah awal kata negative
    unique_words         = set() # set untuk mengumpulkan kata unique dari positive dan negative
    
    for text in positive_data: #  iterasi untuk menjumlah seluruh kata positif yang ditemukan
        words = re.findall(r'\b\w+\b', text)
        for word in words:
            word_freq_positive[word] += 1
            total_words_positive += 1
            unique_words.add(word)
            
    
    for text in negative_data: # iterasi untuk menjumlah seluruh kata negatif yang ditemukan
        words = re.findall(r'\b\w+\b', text)
        for word in words:
            word_freq_negative[word] += 1
            total_words_negative += 1
            unique_words.add(word)
    
    total_unique_words = len(unique_words) # total kata unik
    
    model = {'prior_positive'       : prior_positive,
             'prior_negative'       : prior_negative,
             'word_freq_positive'   : word_freq_positive,
             'word_freq_negative'   : word_freq_negative,
             'total_words_positive' : total_words_positive,
             'total_words_negative' : total_words_negative,
             'total_unique_words'   : total_unique_words}
    
    with open('nb_model.pkl', 'wb') as f: # menyimpan model yang telah dibuat ke dalam file pickle
        pickle.dump(model, f)
    return model

In [21]:
from math import log

def KlasifikasiText(text, model):
    word_freq_positive   = model['word_freq_positive']
    word_freq_negative   = model['word_freq_negative']
    total_words_positive = model['total_words_positive']
    total_words_negative = model['total_words_negative']
    total_unique_words   = model['total_unique_words']
    prior_positive       = model['prior_positive']
    prior_negative       = model['prior_negative']
    words                = re.findall(r'\b\w+\b', text.lower())
    likelihood_positive  = log(prior_positive)
    likelihood_negative  = log(prior_negative)
    
    for word in words:
        likelihood_positive += log((word_freq_positive[word] + 1) / (total_words_positive + total_unique_words))
        likelihood_negative += log((word_freq_negative[word] + 1) / (total_words_negative + total_unique_words))
    
    # Calculate probabilities for each class
    prob_positive = 1 / (1 + pow(2.71828, -likelihood_positive))
    prob_negative = 1 / (1 + pow(2.71828, -likelihood_negative))

    return prob_positive, prob_negative

data_latih_positif  = ['1. Blackpink di coachella 2023 keren bgt!!!!',
                       '2. Suka banget lagu Tally Blackpink',
                       '3. Outfit Willow keren bgt sih',
                       '4. Coachella tahun ini luar biasa',
                       '5. Kapan ya ada acara di indo yang seperti Coachella Wkwkwkwkwkwkwkkwkw',
                       '6. Blackpink comeback soon!', 'Kelas Positif']
data_latih_negative = ['1. Acaranya sebentar banget rasanya :(',
                       '2. Mahal banget tiketnya aslii, males jadinya',
                       '3. Wah lineup Coachella mengecewakan',
                       '4. Cuacanya parah banget. Mana desek - desekan. plis lah Coachella',
                       '5. Padet banget disana, gak berani masuk jadinya',
                       '6. Tiket Coachella sudah habis.', 'Kelas Negatif']

model               = nb_train(data_latih_positif, data_latih_negative)
prob_positive, prob_negative = KlasifikasiText(text_input, model)

In [37]:
text_input = 'Personel Blackpink cantik-cantik'
prob_positive, prob_negative = KlasifikasiText(text_input, model)

# Display probabilities without scientific notation
print('Probability Kelas Positif:', '{:.20f}'.format(prob_positive))
print('Probability Kelas Negatif:', '{:.20f}'.format(prob_negative))

Probability Kelas Positif: 0.00000001709628919546
Probability Kelas Negatif: 0.00000000427407633922
